In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
gdp_df = pd.read_csv('gdp_analysis.csv')
#X = gdp_df.loc[:,['slope', 'y_intercept', 'volatility', 'min_year', 'max_year']]
X = gdp_df.loc[:,['slope', 'y_intercept', 'volatility']]
y = gdp_df.loc[:, 'got_loan']
#y = gdp_df.loc[:, 'got_mentioned']

In [3]:
X.head()

,slope,y_intercept,volatility
0,7.675012,-1.497918e+04,0.540344
1,155.758189,-3.094915e+05,0.370373
2,76.377837,-1.498252e+05,0.568859
3,228.691323,-4.494300e+05,0.597126
4,918.360834,-1.809247e+06,0.385188


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify = y, test_size = 0.2, shuffle = True)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier
import xgboost

In [6]:
rf = RandomForestClassifier(n_estimators = 100, max_depth = 5, criterion = 'entropy')
rf.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=5)

In [7]:
print(accuracy_score(y, rf.predict(X)))
print(accuracy_score(y_train, rf.predict(X_train)))
print(accuracy_score(y_test, rf.predict(X_test)))
#print(roc_auc_score(y_test, rf.predict_proba(X_test)[:,1]))

0.847953216374269
0.8823529411764706
0.7142857142857143


In [8]:
ada = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 2), learning_rate = 0.3,
                        n_estimators = 3)
ada.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2),
                   learning_rate=0.3, n_estimators=3)

In [9]:
print(accuracy_score(y, ada.predict(X)))
print(accuracy_score(y_train, ada.predict(X_train)))
print(accuracy_score(y_test, ada.predict(X_test)))
#print(roc_auc_score(y_test, ada.predict_proba(X_test)[:,1]))

0.7134502923976608
0.7352941176470589
0.6285714285714286


In [10]:
np.mean(y)

0.5672514619883041

In [11]:
ex = ExtraTreesClassifier(n_estimators = 60, max_depth = 2)
ex.fit(X_train, y_train)

ExtraTreesClassifier(max_depth=2, n_estimators=60)

In [12]:
print(accuracy_score(y, ex.predict(X)))
print(accuracy_score(y_train, ex.predict(X_train)))
print(accuracy_score(y_test, ex.predict(X_test)))

0.672514619883041
0.6617647058823529
0.7142857142857143


In [13]:
from sklearn.model_selection import StratifiedKFold

In [24]:
kf = StratifiedKFold(n_splits = 20)
cv = np.zeros((20,2))
i = 0
for train_index, test_index in kf.split(X,y):
    X_train= X.iloc[train_index]
    y_train = y.iloc[train_index]
    X_val = X.iloc[test_index]
    y_val = y.iloc[test_index]
    
    rf = RandomForestClassifier(n_estimators = 60, max_depth = 4, criterion = 'gini', bootstrap = False)
    rf.fit(X_train,y_train)
    
    ex = ExtraTreesClassifier(n_estimators = 100, max_depth = 6)
    ex.fit(X_train,y_train)
    
        #ada = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(max_depth = 2), learning_rate = 0.3,
                        #n_estimators = 10)
        #ada.fit(X_train, y_train)
    
    cv[i,0] = accuracy_score(y_val, rf.predict(X_val))
    cv[i,1] = accuracy_score(y_val, ex.predict(X_val))
    i = i+1
    

In [25]:
np.mean(cv, axis = 0)

array([0.64375   , 0.64791667])